In [1]:
%pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fatihkgg/affectnet-yolo-format")

print("Path to dataset files:", path)

100%|██████████| 258M/258M [00:11<00:00, 22.9MB/s] 

Extracting files...


Path to dataset files: /home/sagemaker-user/.cache/kagglehub/datasets/fatihkgg/affectnet-yolo-format/versions/2


In [6]:
%cp -r /home/sagemaker-user/.cache/kagglehub/datasets/fatihkgg/affectnet-yolo-format/versions/2/YOLO_format .

In [ ]:
from ultralytics import YOLO

# ---------------- Hyper‑parameters -----------------
EPOCHS = 200
BATCH  = 16
IMGSZ  = 640
DEVICE = '0' 
# ---------------------------------------------------

model = YOLO("./runs/train/affectnet_yolov8_tuned2/weights/last.pt")


In [ ]:
results = model.train(
    # — DATA ----------------------------------------------------------
    data='./YOLO_format/data.yaml',      # same splits as before
    imgsz=640,                  # upscale a bit (96→128) but stay GPU‑friendly
    batch=16,                   # 16 GB T4 fits 64 @128px with v8‑l
    single_cls=False,           # keep 8 emotions separate
    cache="ram",                # tiny dataset ⇒ RAM‑cache accelerates

    # — OPTIMISER & SCHEDULER ----------------------------------------
    epochs=200,                 # plenty for convergence; see patience
    optimizer="AdamW",          # good for classification‑heavy tasks
    lr0=0.003,                  # lower than v8 default (0.01) for large net
    lrf=0.01,                   # cosine final LR = lr0*0.01
    momentum=0.9,               # β1 if AdamW (`momentum` still applies)
    weight_decay=0.0001,        # lighter L2 so fine details aren’t over‑smoothed
    cos_lr=True,                # cosine schedule
    warmup_epochs=3,            # stabilise at the start

    # — AUGMENTATION -------------------------------------------------
    hsv_h=0.02, hsv_s=0.7, hsv_v=0.4,  # colour jitter (faces are colour‑sensitive)
    flipud=0.0, fliplr=0.5,            # no vertical flips; keep horizontals
    scale=0.5,                         # random zoom ±50 %
    shear=0.0,                          # avoid geometric warps that distort faces
    close_mosaic=0,                    # disable mosaic for small crops
    mixup=0.0,                          # mixing two faces confuses labels
    dropout=0.25,                      # light dropout in head for generalisation

    # — TRAINING BEHAVIOUR -------------------------------------------
    device='0',                          # GPU id
    workers=8,                         # dataloader threads
    amp=True,                          # mixed‑precision for speed
    freeze=10,                         # lock first ≈10 layers (generic edges)
    resume=True,                      # set True if picking up from last.pt
    project="runs/train",      
    name="affectnet_yolov8_tuned"
)

Ultralytics 8.3.112 🚀 Python-3.12.9 torch-2.5.1 CUDA:0 (Tesla T4, 14918MiB)
engine/trainer: task=detect, mode=train, model=runs/train/affectnet_yolov8_tuned2/weights/last.pt, data=./YOLO_format/data.yaml, epochs=200, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=ram, device=0, workers=8, project=runs/train, name=affectnet_yolov8_tuned22, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=0, resume=runs/train/affectnet_yolov8_tuned2/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.25, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, sav

train: Scanning /home/sagemaker-user/YOLO_format/train/labels.cache... 17101 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17101/17101 [00:00<?, ?it/s]


WARNING ⚠️ train: 29.4GB RAM required to cache images with 50% safety margin but only 11.9/15.3GB available, not caching images
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 359.7±285.4 MB/s, size: 9.1 KB)


val: Scanning /home/sagemaker-user/YOLO_format/valid/labels.cache... 5406 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5406/5406 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



val: Caching images (6.2GB RAM): 100%|██████████| 5406/5406 [00:04<00:00, 1348.41it/s]


Plotting labels to runs/train/affectnet_yolov8_tuned22/labels.jpg... 
optimizer: AdamW(lr=0.003, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0001), 63 bias(decay=0.0)


AssertionError: runs/train/affectnet_yolov8_tuned2/weights/last.pt training to 200 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=runs/train/affectnet_yolov8_tuned2/weights/last.pt'